#DZAKY RAIHAN AHMAD
#1301204316
#MENGGUNAKAN METODE LOGISTIC REGRESSION

In [ ]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import cv2
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from skimage.feature import hog
from skimage import exposure

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

--2023-11-05 03:10:15--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.101.207, 142.250.141.207, 142.251.2.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.101.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   136MB/s    in 0.5s    

2023-11-05 03:10:16 (136 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



# TANPA MENGGUNAKAN FEATURE EXTRACTION

In [ ]:
# Load and preprocess the data
image_paths = []  # List of image file paths
labels = []  # List of corresponding labels (0 for cats, 1 for dogs)

image_size = (150, 150)  # Define the desired image size

for img_path in os.listdir(train_cats_dir):
    image = cv2.imread(os.path.join(train_cats_dir, img_path))
    if image is not None:
        image = cv2.resize(image, image_size)  # Resize the image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale if needed
        image = image / 255.0  # Normalize pixel values (if needed)
        image = image.flatten()  # Flatten the image into a 1D array
        image_paths.append(image)
        labels.append(0)  # 0 represents cats

for img_path in os.listdir(train_dogs_dir):
    image = cv2.imread(os.path.join(train_dogs_dir, img_path))
    if image is not None:
        image = cv2.resize(image, image_size)  # Resize the image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale if needed
        image = image / 255.0  # Normalize pixel values (if needed)
        image = image.flatten()  # Flatten the image into a 1D array
        image_paths.append(image)
        labels.append(1)  # 1 represents dogs

# Convert the image_paths and labels lists to NumPy arrays
X = np.array(image_paths)
y = np.array(labels)

# Split data into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply feature scaling using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define a parameter grid to search
"""
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Different values of C
    'penalty': ['l1', 'l2'],  # L1 and L2 regularization
    'max_iter': [1000],
    'solver': ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga']  # Different solvers
}
"""

# Create a Logistic Regression model
logistic_regression = LogisticRegression(C=0.1, max_iter=1000, penalty='l1', solver= 'saga')

"""
grid_search = GridSearchCV(logistic_regression, param_grid, cv=4)

# Fit the model to the training data
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Make predictions using the trained model
best_model = grid_search.best_estimator_
y_test_pred = best_model.predict(X_test)
"""

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', logistic_regression)
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)


#logistic_regression.fit(X_train, y_train)
#y_pred = logistic_regression.predict(X_test)
# Calculate metrics for the test set
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Test Precision: {precision * 100:.2f}%")
print(f"Test Recall: {recall * 100:.2f}%")
print(f"Test F1: {f1 * 100:.2f}%")

Test Accuracy: 50.50%
Test Precision: 50.79%
Test Recall: 48.26%
Test F1: 49.49%


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

# Load and preprocess the validation data
image_paths = []  # List of image file paths
labels = []  # List of corresponding labels (0 for cats, 1 for dogs)

for img_path in os.listdir(validation_cats_dir):
    image = cv2.imread(os.path.join(validation_cats_dir, img_path))
    if image is not None:
        image = cv2.resize(image, image_size)  # Resize the image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale if needed
        image = image / 255.0  # Normalize pixel values
        image = image.flatten()  # Flatten the image into a 1D array
        image_paths.append(image)
        labels.append(0)  # 0 represents cats

for img_path in os.listdir(validation_dogs_dir):
    image = cv2.imread(os.path.join(validation_dogs_dir, img_path))
    if image is not None:
        image = cv2.resize(image, image_size)  # Resize the image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale if needed
        image = image / 255.0  # Normalize pixel values
        image = image.flatten()  # Flatten the image into a 1D array
        image_paths.append(image)
        labels.append(1)  # 1 represents dogs

# Convert the image_paths and labels lists to NumPy arrays
X_validation = np.array(image_paths)
y_validation = np.array(labels)

# Use your trained Logistic Regression model to make predictions on the validation data
y_validation_pred = logistic_regression.predict(X_validation)

# Calculate metrics for the validation data
accuracy_validation = accuracy_score(y_validation, y_validation_pred)
precision_validation = precision_score(y_validation, y_validation_pred)
recall_validation = recall_score(y_validation, y_validation_pred)
f1_validation = f1_score(y_validation, y_validation_pred)

print(f"Validation Accuracy: {accuracy_validation * 100:.2f}%")
print(f"Validation Precision: {precision_validation * 100:.2f}%")
print(f"Validation Recall: {recall_validation * 100:.2f}%")
print(f"Validation F1: {f1_validation * 100:.2f}%")

Validation Accuracy: 51.30%
Validation Precision: 51.22%
Validation Recall: 54.60%
Validation F1: 52.86%


# MENGGUNAKAN FEATURE EXTRACTION HOG

In [ ]:
# Function to extract HOG features from an image
def extract_hog_features(image):
    # Compute HOG features
    features, hog_image = hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    return features

# Load and preprocess the data
image_paths = []  # List of image file paths
labels = []  # List of corresponding labels (0 for cats, 1 for dogs)

for img_path in os.listdir(train_cats_dir):
    image = cv2.imread(os.path.join(train_cats_dir, img_path))
    if image is not None:
        image = cv2.resize(image, (150, 150))  # Resize the image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale if needed
        features = extract_hog_features(image)  # Extract HOG features
        image_paths.append(features)
        labels.append(0)  # 0 represents cats

for img_path in os.listdir(train_dogs_dir):
    image = cv2.imread(os.path.join(train_dogs_dir, img_path))
    if image is not None:
        image = cv2.resize(image, (150, 150))  # Resize the image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale if needed
        features = extract_hog_features(image)  # Extract HOG features
        image_paths.append(features)
        labels.append(1)  # 1 represents dogs


# Convert the image_paths and labels lists to NumPy arrays
X = np.array(image_paths)
y = np.array(labels)

# Split data into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply feature scaling using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define a parameter grid to search
"""
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Different values of C
    'penalty': ['l1', 'l2'],  # L1 and L2 regularization
    'max_iter': [1000],
    'solver': ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga']  # Different solvers
}
"""

# Create a Logistic Regression model
logistic_regression = LogisticRegression(C=0.1, max_iter=1000, penalty='l1', solver= 'saga')

"""
grid_search = GridSearchCV(logistic_regression, param_grid, cv=4)

# Fit the model to the training data
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Make predictions using the trained model
best_model = grid_search.best_estimator_
y_test_pred = best_model.predict(X_test)
"""

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', logistic_regression)
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)


#logistic_regression.fit(X_train, y_train)
#y_pred = logistic_regression.predict(X_test)
# Calculate metrics for the test set
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Test Precision: {precision * 100:.2f}%")
print(f"Test Recall: {recall * 100:.2f}%")
print(f"Test F1: {f1 * 100:.2f}%")

Test Accuracy: 68.25%
Test Precision: 68.88%
Test Recall: 67.16%
Test F1: 68.01%


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

# Load and preprocess the validation data
image_paths = []  # List of image file paths
labels = []  # List of corresponding labels (0 for cats, 1 for dogs)

for img_path in os.listdir(validation_cats_dir):
    image = cv2.imread(os.path.join(validation_cats_dir, img_path))
    if image is not None:
        image = cv2.resize(image, (150, 150))  # Resize the image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale if needed
        features = extract_hog_features(image)  # Extract HOG features
        image_paths.append(features)
        labels.append(0)  # 0 represents cats

for img_path in os.listdir(validation_dogs_dir):
    image = cv2.imread(os.path.join(validation_dogs_dir, img_path))
    if image is not None:
        image = cv2.resize(image, (150, 150))  # Resize the image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale if needed
        features = extract_hog_features(image)  # Extract HOG features
        image_paths.append(features)
        labels.append(1)  # 1 represents dogs


# Convert the image_paths and labels lists to NumPy arrays
X_validation = np.array(image_paths)
y_validation = np.array(labels)

# Use your trained Logistic Regression model to make predictions on the validation data
y_validation_pred = pipeline.predict(X_validation)

# Calculate metrics for the validation data
accuracy_validation = accuracy_score(y_validation, y_validation_pred)
precision_validation = precision_score(y_validation, y_validation_pred)
recall_validation = recall_score(y_validation, y_validation_pred)
f1_validation = f1_score(y_validation, y_validation_pred)

print(f"Validation Accuracy: {accuracy_validation * 100:.2f}%")
print(f"Validation Precision: {precision_validation * 100:.2f}%")
print(f"Validation Recall: {recall_validation * 100:.2f}%")
print(f"Validation F1: {f1_validation * 100:.2f}%")

Validation Accuracy: 68.10%
Validation Precision: 66.98%
Validation Recall: 71.40%
Validation F1: 69.12%


#MENGGUNAKAN FEATURE EXTRACTION LBP

In [ ]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import cv2
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from skimage.feature import local_binary_pattern
from skimage import exposure

In [ ]:
# Function to extract LBP features from an image
def extract_lbp_features(image):
    radius = 1
    n_points = 8 * radius
    lbp_image = local_binary_pattern(image, n_points, radius, method='uniform')
    lbp_hist, _ = np.histogram(lbp_image.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    lbp_hist = lbp_hist.astype("float")
    lbp_hist /= (lbp_hist.sum() + 1e-6)
    return lbp_hist

# Load and preprocess the data
image_paths = []  # List of image file paths
labels = []  # List of corresponding labels (0 for cats, 1 for dogs)

for img_path in os.listdir(train_cats_dir):
    image = cv2.imread(os.path.join(train_cats_dir, img_path))
    if image is not None:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        features = extract_lbp_features(image)  # Extract LBP features
        image_paths.append(features)
        labels.append(0)  # 0 represents cats

for img_path in os.listdir(train_dogs_dir):
    image = cv2.imread(os.path.join(train_dogs_dir, img_path))
    if image is not None:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        features = extract_lbp_features(image)  # Extract LBP features
        image_paths.append(features)
        labels.append(1)  # 1 represents dogs

# Convert the image_paths and labels lists to NumPy arrays
X = np.array(image_paths)
y = np.array(labels)

# Split data into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply feature scaling using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create a Logistic Regression model
logistic_regression = LogisticRegression(C=0.1, max_iter=1000, penalty='l1', solver='saga')

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', logistic_regression)
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

# Calculate metrics for the test set
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Test Precision: {precision * 100:.2f}%")
print(f"Test Recall: {recall * 100:.2f}%")
print(f"Test F1: {f1 * 100:.2f}%")

Test Accuracy: 63.25%
Test Precision: 62.16%
Test Recall: 68.66%
Test F1: 65.25%


In [ ]:
# Load and preprocess the validation data
image_paths = []  # List of image file paths
labels = []  # List of corresponding labels (0 for cats, 1 for dogs)

for img_path in os.listdir(validation_cats_dir):
    image = cv2.imread(os.path.join(validation_cats_dir, img_path))
    if image is not None:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        features = extract_lbp_features(image)  # Extract LBP features
        image_paths.append(features)
        labels.append(0)  # 0 represents cats

for img_path in os.listdir(validation_dogs_dir):
    image = cv2.imread(os.path.join(validation_dogs_dir, img_path))
    if image is not None:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        features = extract_lbp_features(image)  # Extract LBP features
        image_paths.append(features)
        labels.append(1)  # 1 represents dogs

# Convert the image_paths and labels lists to NumPy arrays
X_validation = np.array(image_paths)
y_validation = np.array(labels)

# Use your trained Logistic Regression model to make predictions on the validation data
y_validation_pred = pipeline.predict(X_validation)

# Calculate metrics for the validation data
accuracy_validation = accuracy_score(y_validation, y_validation_pred)
precision_validation = precision_score(y_validation, y_validation_pred)
recall_validation = recall_score(y_validation, y_validation_pred)
f1_validation = f1_score(y_validation, y_validation_pred)

print(f"Validation Accuracy: {accuracy_validation * 100:.2f}%")
print(f"Validation Precision: {precision_validation * 100:.2f}%")
print(f"Validation Recall: {recall_validation * 100:.2f}%")
print(f"Validation F1: {f1_validation * 100:.2f}%")

Validation Accuracy: 50.00%
Validation Precision: 50.00%
Validation Recall: 100.00%
Validation F1: 66.67%


#MENGGUNAKAN FEATURE EXTRACTION SIFT


In [ ]:
pip install opencv-contrib-python

In [ ]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import cv2
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

In [ ]:
# Function to extract SIFT features from an image
def extract_sift_features(image, sift):
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return keypoints, descriptors

# Load and preprocess the data
image_paths = []  # List of image file paths
labels = []  # List of corresponding labels (0 for cats, 1 for dogs)

for img_path in os.listdir(train_cats_dir):
    image = cv2.imread(os.path.join(train_cats_dir, img_path), cv2.IMREAD_GRAYSCALE)
    if image is not None:
        image_paths.append(image)
        labels.append(0)  # 0 represents cats

for img_path in os.listdir(train_dogs_dir):
    image = cv2.imread(os.path.join(train_dogs_dir, img_path), cv2.IMREAD_GRAYSCALE)
    if image is not None:
        image_paths.append(image)
        labels.append(1)  # 1 represents dogs

# Create SIFT extractor
sift = cv2.xfeatures2d.SIFT_create()

# Extract SIFT features from images
keypoints_list = []
descriptors_list = []

for image in image_paths:
    keypoints, descriptors = extract_sift_features(image, sift)
    keypoints_list.extend(keypoints)
    descriptors_list.extend(descriptors)

# Convert descriptors to a NumPy array
descriptors_array = np.array(descriptors_list)

# Create a KMeans model with a fixed cluster size
desired_length = 128  # Adjust as needed
kmeans = KMeans(n_clusters=desired_length, random_state=42)
kmeans.fit(descriptors_array)

# Create histograms of visual words for training data
X_train_histograms = []

for image in image_paths:
    _, descriptors = extract_sift_features(image, sift)
    cluster_assignments = kmeans.predict(descriptors)
    histogram = np.bincount(cluster_assignments, minlength=desired_length)
    X_train_histograms.append(histogram)

X_train_histograms = np.vstack(X_train_histograms)

# Convert the labels to a NumPy array
y = np.array(labels)

# Split data into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X_train_histograms, y, test_size=0.2, random_state=42)

# Create a Logistic Regression model
logistic_regression = LogisticRegression(C=0.1, max_iter=1000, penalty='l1', solver='saga')

pipeline = Pipeline([
    ('classifier', logistic_regression)
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

# Calculate metrics for the test set
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Test Precision: {precision * 100:.2f}%")
print(f"Test Recall: {recall * 100:.2f}%")
print(f"Test F1: {f1 * 100:.2f}%")

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Test Accuracy: 69.25%
Test Precision: 67.73%
Test Recall: 74.13%
Test F1: 70.78%


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
# Evaluate the model on the validation set

# Load and preprocess the validation data
validation_image_paths = []  # List of image file paths for validation
validation_labels = []  # List of corresponding labels (0 for cats, 1 for dogs) for validation

for img_path in os.listdir(validation_cats_dir):
    image = cv2.imread(os.path.join(validation_cats_dir, img_path), cv2.IMREAD_GRAYSCALE)
    if image is not None:
        validation_image_paths.append(image)
        validation_labels.append(0)  # 0 represents cats

for img_path in os.listdir(validation_dogs_dir):
    image = cv2.imread(os.path.join(validation_dogs_dir, img_path), cv2.IMREAD_GRAYSCALE)
    if image is not None:
        validation_image_paths.append(image)
        validation_labels.append(1)  # 1 represents dogs

# Extract SIFT features from validation images
validation_keypoints_list = []
validation_descriptors_list = []

for image in validation_image_paths:
    keypoints, descriptors = extract_sift_features(image, sift)
    validation_keypoints_list.extend(keypoints)
    validation_descriptors_list.extend(descriptors)

# Convert descriptors to a NumPy array for validation data
validation_descriptors_array = np.array(validation_descriptors_list)

# Create histograms of visual words for validation data
X_validation_histograms = []

for image in validation_image_paths:
    _, descriptors = extract_sift_features(image, sift)
    cluster_assignments = kmeans.predict(descriptors)
    histogram = np.bincount(cluster_assignments, minlength=desired_length)
    X_validation_histograms.append(histogram)

X_validation_histograms = np.vstack(X_validation_histograms)

# Convert the labels to a NumPy array for validation data
y_validation = np.array(validation_labels)

# Predict using the trained model
y_validation_pred = pipeline.predict(X_validation_histograms)

# Calculate metrics for the validation set
validation_accuracy = accuracy_score(y_validation, y_validation_pred)
validation_precision = precision_score(y_validation, y_validation_pred)
validation_recall = recall_score(y_validation, y_validation_pred)
validation_f1 = f1_score(y_validation, y_validation_pred)

# Print validation metrics
print(f"Validation Accuracy: {validation_accuracy * 100:.2f}%")
print(f"Validation Precision: {validation_precision * 100:.2f}%")
print(f"Validation Recall: {validation_recall * 100:.2f}%")
print(f"Validation F1: {validation_f1 * 100:.2f}%")


Validation Accuracy: 68.70%
Validation Precision: 67.03%
Validation Recall: 73.60%
Validation F1: 70.16%


#KESIMPULAN

Kesimpulannya, dengan menggunakan feature extraction dapat meningkatkan akurasi daripada yang tidak menggunakan feature extraction. Namun hal ini juga tergantung metode yang digunakan, bisa dilihat bahwa menggunakan feature extraction LBP tidak mengubah akurasi yang begitu banyak jika dibandingkan dengan tanpa menggunakan feature extraction